In [493]:
import os
import math
import numpy as np
import pandas as pd 

os.chdir(r'/Users/nairutimehta/Desktop/MHCPTCR/PepHF_OrigPep') 

In [494]:
def change_number(fix_1, vary, fix_2):
    """
    Change the num of the vary part.
    """
    start = int(fix_1[-1][24:26]) + 1
    end = int(fix_2[0][24:26]) - 1
    last = vary[0][24:26]
    now = str(start)
    for i in range(len(vary)):
        line = vary[i]
        if line[24:26] == last:
            line = line[:24] + " " + now + line[26:]
        else:
            now = str(int(now) + 1)
            line = line[:24] + " " + now + line[26:]
            last = str(int(last) + 1)
        vary[i] = line
    return vary

In [495]:
def modify_peptide(pdb_name, conformations):
    """
    Select the peptide with given number and modify the format of peptide.
    """
    input_file = pdb_name + '_vary_peptide.pdb'

    with open(input_file,'r') as pdb_file:
        lines = pdb_file.readlines()

    format_1 = 'TER' + ' '*(5 - len(str(conformations))) + str(conformations)
    format_2 = 'TER' + ' '*(5 - len(str(conformations+1))) + str(conformations+1)

    fix_vary_idx = []

    for i, line in enumerate(lines):
        if line[0:8] == format_1:
            vary_begin_idx = i + 1

        if line[0:8] == format_2:
            vary_end_idx = i

        if line.strip() == 'TER':
            fix_begin_idx = i + 1

        if line[0:3] != 'TER' and line[21].isalpha() == False:
            fix_vary_idx.append(i)

    fix_1 = lines[fix_begin_idx:fix_vary_idx[0]]
    vary = lines[vary_begin_idx:vary_end_idx]
    fix_2 = lines[fix_vary_idx[-1]+1:]
    
    vary = change_number(fix_1, vary, fix_2)
    
    result = fix_1+ vary + fix_2

    idx = [i+1 for i in range(len(result))]

    for i in range(len(result)):
        result[i] = result[i][:-1] + '  1.00  0.00' + ' '* 14 + '\n'
        result[i] = result[i][:8] + ' '*(3-len(str(idx[i]))) + str(idx[i]) + result[i][11:]
        result[i] = result[i].replace(result[i][21], "C")

        if result[i][12] != ' ':
            result[i] = result[i][:12] + ' ' + result[i][13:]
        
    result.insert(0,'TER' + ' '*(5 - len(str(conformations))) + str(conformations) +'\n')
    return result


In [497]:
def category(dist):
    """
    The category of distance.
    """
    if dist < 0.5:
        cate = "<0.5"
    elif dist >= 0.5 and dist < 1:
        cate = "[0.5, 1]"
    elif dist >= 1 and dist < 1.5:
        cate = "[1, 1.5]"
    elif dist >= 1.5 and dist < 3:
        cate = "[1.5, 3]"
    elif dist >= 3 and dist < 5:
        cate = "[3, 5]"
    else:
        cate = ">=5"
    return cate

In [498]:
def distance_calculate(pdb_name, conformations):
    """
    Calculate the distance between atoms.
    """
    lines = modify_peptide(pdb_name, conformations)
    carbons = ["CA ","CB ","CG ","CD ","CE ","CZ ","CH ","CD1","CD2","CG1","CG2","CE1","CE2","CE3","CZ1","CZ2","CH1","CH2"]
    atom_name = []
    res_name = []
    res_num = []
    x_coord = []
    y_coord =[]
    z_coord =[]
    for line in lines:
        if line[13:16] in carbons:
            atom_name.append(str(line[13:16]))
            res_name.append(str(line[17:20]))
            res_num.append(int(line[24:26]))
            x_coord.append(float(line[31:38]))
            y_coord.append(float(line[39:46]))
            z_coord.append(float(line[47:54]))

    output = []
    peptide_name = pdb_name + "_" + str(conformations)
    for i in range(0, len(x_coord), 1):
        for j in range(i+1, len(x_coord), 1):
            if res_num[i] != res_num[j]:
                xx = (x_coord[i]-x_coord[j])**2
                yy = (y_coord[i]-y_coord[j])**2
                zz = (z_coord[i]-z_coord[j])**2
                dist = float(math.sqrt(xx+yy+zz))
                cate = category(dist)
                output.append([peptide_name, atom_name[i], res_name[i], res_num[i], atom_name[j], res_name[j], res_num[j], float(dist), cate])
    output = np.array(output)
    return output

In [499]:
pdb_name = '2esv'
conformations_list = [53, 49, 36, 71, 19, 12, 80, 8, 14, 48, 93, 92, 89, 26, 21]

In [500]:

res_all = []
for conformations in conformations_list:
    res = modify_peptide(pdb_name, conformations)
    res_all += res
    
output_file = pdb_name + '_peptide_15conformers.pdb'
with open(output_file, 'w') as f:
    for line in res_all:
        f.write(line)

In [501]:

columns = ["peptide name", "atom name i", "res name i", "res num i", "atom name j", "res name j", "res num j", "distance", "distance category"]


file_name = pdb_name + "_peptide_distanceAP.xlsx"
with pd.ExcelWriter(file_name) as writer:
    for conformations in conformations_list:
        res = distance_calculate(pdb_name, conformations)
        conformation_name = pdb_name + "_peptide_" + str(conformations)
        pd.DataFrame(res, columns=columns).to_excel(writer, sheet_name=conformation_name, index=None)
excel_file = pd.read_excel(file_name,sheet_name=None)
file_name2 = pdb_name + "_peptide_distanceAP" + "_compile.xlsx"
df2 = pd.concat(excel_file.values())
df2.to_excel(file_name2,index=None)
        

In [502]:
def category(dist):
    """
    The category of distance.
    """
    if dist < 0.5:
        cate = "<0.5"
    elif dist >= 0.5 and dist < 1:
        cate = "[0.5, 1]"
    elif dist >= 1 and dist < 1.5:
        cate = "[1, 1.5]"
    elif dist >= 1.5 and dist < 3:
        cate = "[1.5, 3]"
    elif dist >= 3 and dist < 5:
        cate = "[3, 5]"
    else:
        cate = ">=5"
    return cate

def distance_calculate2(pdb_name, lines):
    """
    Calculate the distance between atoms.
    """
    carbons = ["CA ","CB ","CG ","CD ","CE ","CZ ","CH ","CD1","CD2","CG1","CG2","CE1","CE2","CE3","CZ1","CZ2","CH1","CH2"]
    atom_name = []
    res_name = []
    res_num = []
    x_coord = []
    y_coord =[]
    z_coord =[]
    for line in lines:
        if line[13:16] in carbons:
            atom_name.append(str(line[13:16]))
            res_name.append(str(line[17:20]))
            res_num.append(int(line[24:26]))
            x_coord.append(float(line[31:38]))
            y_coord.append(float(line[39:46]))
            z_coord.append(float(line[47:54]))

    output = []

    for i in range(0, len(x_coord), 1):
        for j in range(i+1, len(x_coord), 1):
            if res_num[i] != res_num[j]:
                xx = (x_coord[i]-x_coord[j])**2
                yy = (y_coord[i]-y_coord[j])**2
                zz = (z_coord[i]-z_coord[j])**2
                dist = math.sqrt(xx+yy+zz)
                cate = category(dist)
                output.append([pdb_name, atom_name[i], res_name[i], res_num[i], atom_name[j], res_name[j], res_num[j], dist, cate])
    output = np.array(output)
    return output


In [503]:
pdb1_name = '2esv'
input_file = pdb1_name + "_peptide.pdb"
with open(input_file,'r') as pdb_file:
    lines=pdb_file.readlines()

res = distance_calculate2(pdb1_name, lines)

columns = ["peptide name", "atom name i", "res name i", "res num i", "atom name j", "res name j", "res num j", "distance", "distance category"]

file_name3 = pdb1_name + "_peptide_distance.xlsx"
pd.DataFrame(res, columns=columns).to_excel(file_name3, index=None)

In [504]:
f1 = pd.read_excel(file_name2)
f2 = pd.read_excel(file_name3)
f3 = f2.append(f1)
f3.to_excel(pdb_name + "_peptide_distance" + "_compile.xlsx", index = False)

/var/folders/n2/kylnwjhs1q12239qh52h4qx00000gn/T/ipykernel_22578/4109565232.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  f3 = f2.append(f1)
